# Week 9 과제 1

**잎 사진을 통한 질병 분류**

*   데이터: 21개의 클래스로 구분된 약 20000장의 이미지
><img src="https://drive.google.com/uc?id=1YQkxnNy61Gyi3Gp6ylCKeS72BVruJXr_" width="700" height="500"> 
*   데이터 전처리
    *   전체 데이터를 train,validation,test로 분할해주세요
    *   저는 6:2:2로 분할했는데, 비율은 바꾸셔도 무방합니다.
   
    
*   학습 진행 방향
    *   Baseline 모델(pre-trained model 사용X) 구축
    *   Pre-trained 모델 사용
    *   Baseline과 ResNet50 모델의 성능을 비교해주세요 ~!

In [ ]:
# gdrive에 mount
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# 경로 설정
import os
os.chdir('/content/gdrive/My Drive/tobigs')

In [ ]:
# !mkdir -p ./tobigs_week9_plant_leaf_data
!unzip tobigs_week9_plant_leaf_data.zip

## 0. 데이터 분할

* 데이터 분할을 위한 디렉토리 생성

In [ ]:
import os
import shutil
 
original_dataset_dir = './tobigs_week9_plant_leaf'   
classes_list = os.listdir(original_dataset_dir) 
 
base_dir = './splitted' 
os.mkdir(base_dir)
 
train_dir = os.path.join(base_dir, 'train') 
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for cls in classes_list:     
    os.mkdir(os.path.join(train_dir, cls))
    os.mkdir(os.path.join(validation_dir, cls))
    os.mkdir(os.path.join(test_dir, cls))

* 데이터 분할과 클래스별 데이터 수 확인

In [ ]:
import math
 
for cls in classes_list:
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path)
 
    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)
    
    train_fnames = fnames[:train_size]
    print("Train size(",cls,"): ", len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print("Validation size(",cls,"): ", len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    test_fnames = fnames[(train_size+validation_size):(validation_size + train_size +test_size)]

    print("Test size(",cls,"): ", len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst)


In [ ]:
original_dataset_dir = './tobigs_week9_plant_leaf'   
base_dir = './splitted' 
train_dir = os.path.join(base_dir, 'train') 
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
cnt = 0
for dirpath, dirnames, filenames in os.walk(train_dir):
    cnt += len(filenames)
print('train dataset count:', cnt)

train dataset count: 11503


In [ ]:
cnt = 0
for dirpath, dirnames, filenames in os.walk(validation_dir):
    cnt += len(filenames)
print('val dataset count:', cnt)

val dataset count: 3830


In [ ]:
cnt = 0
for dirpath, dirnames, filenames in os.walk(test_dir):
    cnt += len(filenames)
print('test dataset count:', cnt)

test dataset count: 3830


## 1. 베이스라인 모델을 구축해 주세요
* Pre-Trained Model을 사용하지 않고 직접 모델을 구축해 주세요 !


# 새 섹션

In [ ]:
import torch
import os
 
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
BATCH_SIZE = 256 
EPOCH = 30 

In [ ]:
torch.cuda.is_available()

True

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder 
 
transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()]) 
train_dataset = ImageFolder(root='./splitted/train', transform=transform_base) 
val_dataset = ImageFolder(root='./splitted/val', transform=transform_base)

In [ ]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


* 베이스라인 모델 설계하기

In [ ]:
len(classes_list)

21

In [ ]:
print('train data count: {}'.format(len(train_dataset))) # train data 크기
print('input image size: {}'.format(train_dataset[0][0].shape)) # 인풋 이미지 크기
print('total batch count: {}'.format(len(train_loader))) # 총 배치 갯수

train data count: 11503
input image size: torch.Size([3, 64, 64])
total batch count: 45


In [ ]:
len(train_loader)

45

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
class Net(nn.Module): 
  
    def __init__(self): 
    
        super(Net, self).__init__() 

        # input: (3, 64, 64)
        # conv: (15, 64, 64)
        # pool: (15, 32, 32)
        self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(3, 15, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = torch.nn.Linear(15*32*32, 21, bias=True) 
    
    def forward(self, x):  
        out = self.layer1(x)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        # return F.log_softmax(x, dim=1)  
        return out


In [ ]:
model_base = Net().to(DEVICE) 

In [ ]:
criterion = torch.nn.CrossEntropyLoss().to(DEVICE)   # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = optim.Adam(model_base.parameters(), lr=0.001) 

* 모델 학습을 위한 함수
* 모델 학습, 평가를 위한 가이드라인 코드입니다. 꼭 이 코드를 사용하지는 않으셔도 됩니다 !!

In [ ]:
def train(model, train_loader, optimizer):
    # model."코드 작성해주세요"  
    # [inputs, labels]의 목록인 train_loader로부터 입력을 받은 후
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE) 
        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()
        # 순전파 + 역전파 + 최적화 
        output = model_base(data)
        loss =  criterion(output, target) #Cross Entropy Loss 사용했습니다
        loss.backward()
        optimizer.step() 

* 모델 평가를 위한 함수

In [ ]:
def evaluate(model, test_loader):
    # model."코드 작성해주세요" 
    test_loss = 0 
    correct = 0   
    
    with torch.no_grad(): 
        for data, target in test_loader:  
            data, target = data.to(DEVICE), target.to(DEVICE)  
            output = model(data) 
            
            test_loss += F.cross_entropy(output,target, reduction='sum').item() 
 
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item() 
   
    test_loss /= len(test_loader.dataset) 
    test_accuracy = 100. * correct / len(test_loader.dataset) 
    return test_loss, test_accuracy  

* 모델 학습을 실행하기

In [ ]:
train_loss, train_acc = evaluate(model_base, train_loader)
train_loss, train_acc

In [ ]:
import time
import copy
 
def train_baseline(model ,train_loader, val_loader, optimizer,num_epochs = 30):
    best_acc = 0.0  
    best_model_wts = copy.deepcopy(model.state_dict()) 
 
    for epoch in range(1, num_epochs + 1):
        since = time.time()  
        train(model_base, train_loader, optimizer)
        train_loss, train_acc = evaluate(model_base, train_loader)
        val_loss, val_acc = evaluate(model_base, val_loader)
        
        if val_acc > best_acc: 
            best_acc = val_acc 
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since 
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))   
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) 

    model.load_state_dict(best_model_wts)  
    return model
 

base = train_baseline(model_base ,train_loader, val_loader, optimizer,1)  	
torch.save(base,'baseline.pt')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


-------------- epoch 1 ----------------
train Loss: 0.8934, Accuracy: 73.08%
val Loss: 0.9402, Accuracy: 71.17%
Completed in 5m 29s


## 2. Transfer Learning 모델 학습

* Transfer Learning을 위한 준비
* Transfer Learning이 익숙하지 않으신 분들은 PyTorch에서 제공하는 https://9bow.github.io/PyTorch-tutorials-kr-0.3.1/beginner/transfer_learning_tutorial.html 을 참고하세요 :)

In [100]:

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [117]:
data_transforms = {
    'train': transforms.Compose([transforms.Resize([64,64]), 
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ]),
    
    'val': transforms.Compose([transforms.Resize([64,64]),  
                               transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
}

train_dataset = ImageFolder(root='./splitted/train', transform=data_transforms['train']) 
val_dataset = ImageFolder(root='./splitted/val', transform=data_transforms['val'])
image_datasets = {'train': train_dataset,
                  'val': val_dataset}

In [118]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

dataloaders = {'train':train_loader, 'val':val_loader}
use_gpu = torch.cuda.is_available()
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### Pre-Trained Model 불러오기
- 저는 ResNet50을 사용했는데, 코랩 기준으로 다른 ResNet계열이나 DenseNet 정도까지는 큰 무리 없이 훈련할 수 있습니다. Unfreeze layer 수가 적으면 다른 모델도 사용할 수 있을 것입니다.
- 한 가지 모델을 선택해서 Transfer Learning을 해 주세요 !!

In [113]:
from torchvision import models
 
resnet = models.resnet50(pretrained=True)  
num_ftrs = resnet.fc.in_features   
resnet.fc = nn.Linear(num_ftrs, 21) # 마지막 fc층 변경
resnet = resnet.to(DEVICE)
 
criterion = nn.CrossEntropyLoss() 
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)

* Pre-Trained Model의 일부 Layer Freeze하기 (resnet 기준입니다 !!)

In [114]:
ct = 0 
for child in resnet.children():  
    ct+= 1  
    if ct < 6: 
        for param in child.parameters():
            param.requires_grad = False

* Fine Tuning을 진행해주세요 ~!

In [119]:
def train_model(model, criterion, optimizer, dataloaders, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [120]:
model_ft = train_model(resnet, criterion, optimizer_ft,dataloaders, num_epochs=1)

Epoch 0/0
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 0.0892 Acc: 0.9738
val Loss: 0.1493 Acc: 0.9520

Training complete in 0m 31s
Best val Acc: 0.951958


In [122]:
torch.save(model_ft,'resnet.pt')

## 모델 평가

* 모델 평가를 위해서는 평가 데이터 또한 전처리를 해주어야 합니다.
* validation 데이터와 동일하게 전처리를 해 주세요 ~!!

* 베이스라인 모델 평가를 위한 전처리하기

In [123]:
transform_base = transforms.Compose([transforms.Resize([64,64]),
                                     transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_base = ImageFolder(root='./splitted/test',transform=transform_base)  
test_loader_base = torch.utils.data.DataLoader(test_base, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


* Transfer Learning모델 평가를 위한 전처리하기

In [124]:
transform_resNet = transforms.Compose([
        transforms.Resize([64,64]),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    ])
    
test_resNet = ImageFolder(root='./splitted/test', transform=transform_resNet) 
test_loader_resNet = torch.utils.data.DataLoader(test_resNet,batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### 성능 평가하기
* 저는 여기서 accuracy만을 평가했지만, 분류 모델이기에 다양한 방법의 평가가 가능합니다.
* Confusion Matrix를 이용한 비교도 가능하고, 한 작물에 해당하는 클래스가 여러개인 다중 분류에서 F1-score를 계산하는것도 의미가 있을 것입니다. 
* 다양한 시도를 하시는 분께 가산점 드리겠습니다 :):)

* 베이스라인 모델 성능 평가하기

In [126]:
def evaluate(model, test_loader):
    # model."코드 작성해주세요" 
    test_loss = 0 
    correct = 0   
    
    with torch.no_grad(): 
        for data, target in test_loader:  
            data, target = data.to(DEVICE), target.to(DEVICE)  
            output = model(data) 
            test_loss += F.cross_entropy(output,target, reduction='sum').item() 
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item() 
   
    test_loss /= len(test_loader.dataset) 
    test_accuracy = 100. * correct / len(test_loader.dataset) 
    return test_loss, test_accuracy  

In [127]:
baseline=torch.load('baseline.pt') 
baseline.eval()  
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('test acc:  ', test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


test acc:   42.34986945169713


* Transfer Learning 모델 성능 평가하기

In [128]:
resnet = torch.load('resnet.pt') 
resnet.eval()  
test_loss, test_accuracy = evaluate(resnet, test_loader_resNet)

print('test acc:  ', test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


test acc:   95.74412532637076


* 두 모델의 성능을 비교 평가하는 설명을 작성해주세요 ~!!

우선 트레이닝 시간관계상 두 모델 다 에폭을 1로 돌렸기에 비교하기에 무리가 있을 수 있으나,
전이학습 모델은 에폭을 1로 하였음에도 불구하고 정확도가 95퍼센트를 넘는 것을 볼 수 있었습니다. 
반면 base model은 40퍼센트 정도에 머무르고 있습니다.